In [1]:
#Initialise all pre-requisite libraries and data
import pandas as pd
import calendar

original_data = pd.read_excel('d:\\Users\\Aturoh\\Documents\\Economics\\Government_Securities_in_Excel.xlsx')
#data = pd.read_excel('d:\\Users\\Aturoh\\Documents\\Economics\\Government_Securities_in_Excel.xlsx')

In [2]:
#Converting the 'auctionDate' column to datetime format
original_data['auctionDate'] = pd.to_datetime(original_data['auctionDate'], dayfirst = True)

In [3]:
#Defining the Timeline of the data needed.
data = original_data[original_data['auctionDate'].dt.year.between(2020, 2025)]

In [4]:
#Filtering the dataset by Tenor duration and year 
dataset = {}
dataset['long'] = data[data['tenor'].astype(str).str.contains('364')]
dataset['medium'] = data[data['tenor'].astype(str).str.contains('182')]
dataset['short'] = data[data['tenor'].astype(str).str.contains('91')]

tenors = ['long', 'medium', 'short']
for year in range(2020, 2026):
    for tenor in tenors:
        base = dataset[tenor]
        
        df = dataset[f'{tenor}_{year}'] = base[base['auctionDate'].dt.year == year]
        grouped = df.groupby(df['auctionDate'].dt.month)['rate'].mean().reset_index()
        grouped['investment'] = 100000
        grouped.columns = ['month', 'calculatedRate', 'investment']
        dataset[f'{tenor}_{year}_avg'] = grouped        

In [5]:
#validation and testing 
dataset['long_2024_avg']

,month,calculatedRate,investment
0,1,9.969500,100000
1,2,19.000000,100000
2,3,21.246000,100000
3,4,20.700000,100000
4,5,20.695000,100000
5,6,21.212500,100000
6,7,22.100000,100000
7,8,21.394500,100000
8,9,19.176667,100000
9,10,20.257000,100000


In [6]:
#appending the yearly(Long) tenor Tbills into a single dataset.
a = 0
sumed = []
for year in range(2020, 2026):
    for month in range(0, 12):
        df = dataset[f'long_{year}_avg'] 
        if year == 2020:
            s_p1 = df.loc[0, 'investment']
            df_old = dataset[f'long_2020_{month}'] = df
            df_old['year'] = year
            sums = df_old.copy()
            
        if year > 2020 and month == 0:
            df_prev = dataset[f'long_{year-1}_0']
            new_df = dataset[f'long_{year}_{month}'] = dataset[f'long_{year}_avg'].copy()
            new_df.loc[month, 'investment'] = (df_prev.loc[month, 'investment']* (df_prev.loc[month, 'calculatedRate']/100)) + df_prev.loc[month, 'investment']
            new_df['investment'] = new_df.loc[0, 'investment']
            new_df['year'] = year

            sums = new_df.copy()
            
        if year > 2020 and month > 0:
            df_then = dataset[f'long_{year-1}_{month}'].copy()
            df_then["calculatedRate"] = dataset[f'long_{year-1}_avg']["calculatedRate"].copy()

            df_now = dataset[f'long_{year}_{month}'] = df_then.copy()
            df_now["calculatedRate"] = dataset[f'long_{year}_avg']["calculatedRate"].copy()

            df_now['investment'] = df_then['investment'].iloc[month].copy()
            s_p3 = df_then['investment'].iloc[month] + ((df_then['calculatedRate'].iloc[month]/100)* df_then['investment'].iloc[month])
            df_now['investment'].iloc[month:] = s_p3.copy()
            df_now['year'] = year
            sums = df_now.copy()
            
    
        sumed.append(sums)

f = pd.concat(sumed, ignore_index=True)


C:\Users\user\AppData\Local\Temp\ipykernel_6728\1344894504.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '105596.66666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.loc[month, 'investment'] = (df_prev.loc[month, 'investment']* (df_prev.loc[month, 'calculatedRate']/100)) + df_prev.loc[month, 'investment']
C:\Users\user\AppData\Local\Temp\ipykernel_6728\1344894504.py:31: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, lik

f

In [7]:
#data validation and testing
dataset["long_2024_5"]

,month,calculatedRate,investment,year
0,1,9.969500,120682.924363,2024
1,2,19.000000,120682.924363,2024
2,3,21.246000,120682.924363,2024
3,4,20.700000,120682.924363,2024
4,5,20.695000,120682.924363,2024
5,6,21.212500,130305.376199,2024
6,7,22.100000,130305.376199,2024
7,8,21.394500,130305.376199,2024
8,9,19.176667,130305.376199,2024
9,10,20.257000,130305.376199,2024


In [8]:
#Appending computed data for 2020 to 2025 
fa = []
for years in range(2020, 2026):
        fa.append(dataset[f'long_{years}_0'])
fb = pd.concat(fa)
fb

,month,calculatedRate,investment,year
0,1,5.596667,100000.000000,2020
1,2,6.121600,100000.000000,2020
2,3,4.950000,100000.000000,2020
3,4,4.046667,100000.000000,2020
4,5,3.929950,100000.000000,2020
...,...,...,...,...
1,2,19.375000,132120.777705,2025
2,3,18.945000,132120.777705,2025
3,4,19.615000,132120.777705,2025
4,5,22.855000,132120.777705,2025


In [9]:
#Data exporting
#fb.to_csv('tbilljan5.csv', index = False)